<a href="https://colab.research.google.com/github/komo135/tensorflow-rl/blob/master/ipynb/dqn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Google ドライブをマウントするには、このセルを実行してください。
from google.colab import drive
drive.mount('/content/drive')

%cd drive/My Drive

Mounted at /content/drive
/content/drive/My Drive


In [1]:
%tensorflow_version 2.x
import tensorflow as tf

TensorFlow 2.x selected.


In [4]:
%cd /content/drive/My Drive

try:
  import imp
  imp.reload(dqn)
except:
  import dqn

agent = dqn.Agent(spread=10, pip_cost=1000, leverage=500, min_lots=0.01, assets=10000, available_assets_rate=0.4,
                 restore=not True, step_size=96, n=4, lr=1e-3)

/content/drive/My Drive


In [0]:
%cd /content
agent.run()

action probability: buy=0.5833333333333334, sell=0.4166666666666667, hold=0.0
epoch: 6005, total assets:[618.77298861], growth_rate:[0.0618773]

action probability: buy=0.5520833333333334, sell=0.4479166666666667, hold=0.0
epoch: 6010, total assets:[917.14544573], growth_rate:[0.09171454]

action probability: buy=0.4895833333333333, sell=0.5104166666666666, hold=0.0
epoch: 6015, total assets:[5042.08543726], growth_rate:[0.50420854]

action probability: buy=0.4791666666666667, sell=0.5208333333333334, hold=0.0
epoch: 6020, total assets:[2143.93645425], growth_rate:[0.21439365]

action probability: buy=0.3958333333333333, sell=0.6041666666666666, hold=0.0
epoch: 6025, total assets:[4794.14178935], growth_rate:[0.47941418]

action probability: buy=0.4375, sell=0.5625, hold=0.0
epoch: 6030, total assets:[3704.30101049], growth_rate:[0.3704301]

action probability: buy=0.5416666666666666, sell=0.4583333333333333, hold=0.0
epoch: 6035, total assets:[1224.91217274], growth_rate:[0.12249122]


In [12]:
import numpy as np

def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

# agent.test(spread=10, pip_cost=1000, los_cut=150,test_data=not True)
for _ in range(1):
  tree_idx, replay = agent.memory.sample(128)

  states = np.array([a[0][0] for a in replay], np.float32)
  new_states = np.array([a[0][3] for a in replay], np.float32)
  actions = np.array([a[0][1] for a in replay])
  rewards = np.array([a[0][2] for a in replay], np.float32).reshape((-1, 1))

  with tf.GradientTape() as tape:
    q = agent.model(states)
    target_q = agent.target_model(new_states).numpy()
    arg_q = agent.model(new_states).numpy()
    random = np.random.rand(actions.shape[0])
    arg_q = np.argmax(arg_q, 1)
    # arg_q = np.array([np.argmax(arg_q[i]) if random[i] > 0.1 else np.random.randint(arg_q.shape[1]) for i in
    #                   range(arg_q.shape[0])])

    q_backup = q.numpy()

    for i in range(len(rewards)):
        # q_backup[i, actions[i]] = rewards[i] if I < 1010 and not self.restore else rewards[i] + 0.2 * target_q[i, np.argmax(arg_q[i])]
        q_backup[i, actions[i]] = rewards[i] + 0.1 * target_q[i, arg_q[i]]
    mse = tf.reduce_mean(tf.reduce_sum(tf.abs(q_backup - q) ** 1.5, -1))

  ae = np.array([sum(i) for i in np.abs(q_backup - q.numpy())])
  agent.memory.batch_update(tree_idx, ae)

  gradients = tape.gradient(mse, agent.model.trainable_variables)
  # gradients = [(tf.clip_by_value(grad, -10.0, 10.0))
  #                                   for grad in gradients]
  # agent.model.optimizer.apply_gradients(zip(gradients,agent.model.trainable_variables))
  # print(np.mean(ae))
  # print(q[0:5])
  print(mse)
print(q)
# print(np.mean(rewards)))

tf.Tensor(2311.3635, shape=(), dtype=float32)
tf.Tensor(
[[ 0.7859247   4.155469  ]
 [ 4.4689612   6.6620235 ]
 [ 1.2443707   2.413346  ]
 [ 3.79215     4.0591335 ]
 [-0.716648    2.4889789 ]
 [-6.3182917   4.4925194 ]
 [ 4.7520494   7.9672537 ]
 [ 2.6438003  11.005211  ]
 [-2.010398    8.37055   ]
 [ 0.17102927  7.492281  ]
 [ 2.652239   -2.0900266 ]
 [ 0.76420116  7.386567  ]
 [ 3.4761865   4.354776  ]
 [ 0.07354951  2.8591566 ]
 [ 0.59502745  3.941544  ]
 [ 3.8503215   3.1678262 ]
 [ 3.4350464   0.29605973]
 [ 1.3494759   4.1802373 ]
 [ 2.4065912   3.5978742 ]
 [-2.692646    1.9671077 ]
 [ 1.3157861   4.689899  ]
 [ 3.0772038   1.8700938 ]
 [ 2.5227857   5.1533766 ]
 [ 2.7718573   6.5993414 ]
 [-2.7011666   4.019364  ]
 [ 2.8157587   1.4180268 ]
 [ 5.7640915   4.091986  ]
 [ 1.675692   -3.1225975 ]
 [-1.4816957   3.3747334 ]
 [ 2.6287463   0.03323746]
 [ 4.161395    2.8723898 ]
 [ 7.046205    3.111075  ]
 [ 2.522009    0.60588837]
 [-3.5432127   7.6195083 ]
 [ 2.2942631   2.6658673 

In [14]:
rewards

array([[-215.18],
       [-211.87],
       [-208.76],
       [ 202.29],
       [ 137.49],
       [  67.71],
       [-214.43],
       [-212.78],
       [-165.  ],
       [-215.05],
       [ 213.64],
       [ 280.42],
       [-211.45],
       [-218.48],
       [ 311.39],
       [ 166.68],
       [  95.4 ],
       [-222.73],
       [-180.55],
       [-204.34],
       [-199.12],
       [ 191.17],
       [-178.94],
       [ 164.71],
       [ 343.96],
       [ 285.98],
       [-373.04],
       [-211.11],
       [-123.13],
       [-219.35],
       [ 181.39],
       [-244.66],
       [ 187.48],
       [  67.67],
       [ 214.78],
       [  99.77],
       [-130.23],
       [-105.51],
       [  88.98],
       [-125.27],
       [  86.18],
       [ 184.47],
       [-174.65],
       [-153.26],
       [  99.27],
       [-125.61],
       [ -45.14],
       [-131.8 ],
       [-181.81],
       [ 174.81],
       [  35.41],
       [-156.69],
       [ -96.33],
       [  71.29],
       [-152.3 ],
       [ 2

In [0]:
!pip install ta

import numpy as np
import pandas as pd
import ta
from sklearn.preprocessing import MinMaxScaler

In [0]:
def gen_data(file_path="gbpjpy15.csv"):
    try:
        print("load file")
        df = pd.read_csv(file_path)
    except:
        print("Use 'python gen_data.py 'file_path''")
        return

    df["Close1"] = df["Close"] * 100
    ma = np.array(ta.trend.ema(df["Close1"], 7) - ta.trend.ema(df["Close1"], 4)).reshape((-1,1))
    ma2 = np.array(df["Close1"] - ta.trend.ema(df["Close1"], 7)).reshape((-1,1))
    b1 = np.array(ta.volatility.bollinger_hband(df["Close1"]) - df["Close1"]).reshape((-1,1))
    b2 = np.array(ta.volatility.bollinger_lband(df["Close1"]) - df["Close1"]).reshape((-1,1))
    macd = np.array(ta.trend.macd_diff(df["Close1"])).reshape((-1,1))
    rsi = np.array(ta.momentum.rsi(df["Close"]) - ta.momentum.rsi(df["Close"], 7)).reshape((-1,1))
    stoch = np.array(ta.momentum.stoch_signal(df["High"], df["Low"], df["Close"]) - ta.momentum.stoch(df["High"], df["Low"], df["Close"])).reshape((-1,1))

    x = np.concatenate([ma, macd, rsi, stoch], -1)

    y = np.array(df[["Open"]])
    atr = np.array(ta.volatility.average_true_range(df["High"], df["Low"], df["Close"]))
    high = np.array(df[["High"]])
    low = np.array(df[["Low"]])

    print("gen time series data")
    gen = tf.keras.preprocessing.sequence.TimeseriesGenerator(x, y, 30)
    x = []
    y = []
    for i in gen:
        x.extend(i[0].tolist())
        y.extend(i[1].tolist())
    x = np.asanyarray(x)[100:]
    y = np.asanyarray(y)[100:]
    atr = atr[-len(y):].reshape((-1, 1))
    scale_atr = atr
    high = high[-len(y):].reshape((-1, 1))
    low = low[-len(y):].reshape((-1, 1))

    np.save("x", x)
    np.save("target", np.array([y, atr, scale_atr, high, low]))

    print("done\n")


In [0]:
%cd /content/drive/My Drive
gen_data()

In [0]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

In [0]:
a1 = np.argmax(q,1)

In [0]:
q1 = np.abs(q) / np.sum(np.abs(q),1).reshape((-1,1)) * (np.abs(q) / q)
q1 += .05 * np.random.randn(q1.shape[0], q1.shape[1])
a2 = np.argmax(q1,-1)

In [0]:
np.mean(a1 == a2)

In [0]:
tree_idx, replay = agent.memory.sample(128)

In [0]:
3 in [1,2]